# Ch09. 순환 신경망 (Recurrent Neural Network)

# v06. 글자 단위 RNN (Char RNN)

- 지금까지 배운 RNN은 전부 입력과 출력의 단위가 **단어 벡터**였다.
- 하지만 입출력의 단위를 단어 레벨(word-level)에서 글자 레벨(character-level)로 변경하여 RNN을 구현할 수 있다.
- 어려운 내용이 아니라 입, 출력의 단위를 단어에서 글자로 바꿨을 뿐이다.

$\quad$ ![](https://wikidocs.net/images/page/48649/char_rnn1.PNG)

- 이번 챕터에서는 케라스를 사용하여 글자 단위 RNN(Char RNN)을 구현해본다.

<br>

## 6.1 글자 단위 RNN 언어 모델 (Char RNNLM)

- 이전 시점의 예측 글자를 다음 시점의 입력으로 사용하는 글자 단위 RNN 언어 모델을 구현해보자.
- 앞서 배운 단어 단위 RNN 언어 모델과 다른 점 : 단어 단위가 아니라 글자 단위를 입, 출력으로 사용  
$\rightarrow$ 임베딩층(embedding layer)을 사용하지 않는다.
- 여기서는 언어 모델의 훈련 과정과 테스트 과정의 차이를 이해하는 데 초점을 둔다.

<br>

### 6.1.1 고전 소설 데이터셋

- [다운로드 링크](http://www.gutenberg.org/files/11/11-0.txt)

- 고전 소설들은 저작권에 보호받지 않으므로 무료로 쉽게 다운받을 수 있는 좋은 훈련 데이터이다.
- 위의 링크에서 '이상한 나라의 앨리스(Alice's Adventures in Wonderland)'라는 소설을 다운로드한다.
- 우선 파일을 불러오고 간단한 전처리를 수행한다.

<br>

### 6.1.2 데이터에 대한 이해와 전처리

#### 6.1.2.1 필요 라이브러리 임포트

In [71]:
%tensorflow_version 2.x

import tensorflow as tf
tf.__version__

'2.1.0'

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

<br>

#### 6.1.2.2 데이터셋 불러오기

In [ ]:
f = open('11-0.txt', 'rb')
lines = []

for line in f:
    line = line.strip() # strip() : \r, \n 제거
    line = line.lower()
    line = line.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(line) > 0:
        lines.append(line)

f.close()

In [74]:
lines[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  you may copy it, give it away or',
 're-use it under the terms of the project gutenberg license included',
 'with this ebook or online at www.gutenberg.org']

<br>

#### 6.1.2.3 문자열 통합

- 각 원소는 문자열로 구성되어 있다.
- 특별히 의미있게 문장 토큰화가 된 상태는 아니다.
- 이를 하나의 문자열로 통합하자.

In [75]:
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수 : %d' % len(text))

문자열의 길이 또는 총 글자의 개수 : 159612


In [76]:
print(text[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you may copy it, g


<br>

#### 6.1.2.4 글자 집합 생성

- 이 문자열은 어떤 글자들로 구성되어져 있을까?
- 이제 이 문자열로부터 글자 집합을 만든다.
- 기존에는 중복을 제거한 단어들의 모음인 단어 집합(vocabulary)을 만들었으나, 이번에 만들 집합은 단어 집합이 아니라 글자 집합이다.

In [77]:
char_vocab = sorted(list(set(text)))
vocab_size = len(char_vocab)

print('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 57


- 영어가 훈련 데이터일 때 대부분의 글자 집합의 크기가 단어 집합을 사용했을 경우보다 집합의 크기가 현저히 작다는 특징이 있다.
- 아무리 훈련 코퍼스에 수십만 개 이상의 많은 영어 단어가 존재한다고 하더라도, 영어 단어를 표현하기 위해 글자 집합에 포함되는 글자는 26개의 알파벳뿐이기 때문이다.
- 만약 훈련 데이터의 알파벳이 대, 소문자가 구분된 상태라고 하더라도 모든 영어 단어는 52개의 알파벳으로 표현 가능하다.

<br>

#### 6.1.2.5 글자 집합 인덱스 부여 : `char_to_index`

- 어떤 방대한 양의 텍스트라도 집합의 크기를 적게 가져갈 수 있다는 것은 구현과 테스트를 굉장히 쉽게 할 수 있다는 이점을 가진다.
- 그러므로 RNN의 동작 매커니즘 이해를 위한 토이 프로젝트로 굉장히 많이 사용된다.
- 글자 집합에 인덱스를 부여하고 전부 출력해보자.

In [78]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '@': 27, '[': 28, ']': 29, '_': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56}


- 인덱스 0부터 28까지는 공백을 포함한 각종 구두점, 특수 문자가 존재한다.
- 인덱스 29부터 54까지는 a부터 z까지 총 26개의 알파벳 소문자가 글자 집합에 포함되어져 있다.

<br>

#### 6.1.2.6 `index_to_char`

- 이제 반대로 인덱스로부터 글자를 리턴하는 `index_to_char`를 만든다.

In [79]:
index_to_char = {}

for key, value in char_to_index.items():
    index_to_char[value] = key

print(index_to_char)

{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: "'", 7: '(', 8: ')', 9: '*', 10: ',', 11: '-', 12: '.', 13: '/', 14: '0', 15: '1', 16: '2', 17: '3', 18: '4', 19: '5', 20: '6', 21: '7', 22: '8', 23: '9', 24: ':', 25: ';', 26: '?', 27: '@', 28: '[', 29: ']', 30: '_', 31: 'a', 32: 'b', 33: 'c', 34: 'd', 35: 'e', 36: 'f', 37: 'g', 38: 'h', 39: 'i', 40: 'j', 41: 'k', 42: 'l', 43: 'm', 44: 'n', 45: 'o', 46: 'p', 47: 'q', 48: 'r', 49: 's', 50: 't', 51: 'u', 52: 'v', 53: 'w', 54: 'x', 55: 'y', 56: 'z'}


<br>

#### 6.1.2.7 훈련 데이터 구성

- 훈련 데이터 구성을 위한 간소화된 예를 들어보자.
- 훈련 데이터에 "apple"이라는 시퀀스가 있고, 입력 시퀀스의 길이, 즉 샘플의 길이를 4라고 한다면 입력 시퀀스와 예측해야 하는 출력 시퀀스는 다음과 같이 구성된다.

```python
# Example) 샘플의 길이가 4라면 4개의 입력 글자 시퀀스로부터 4개의 출력 글자 시퀀스를 예측. 즉, RNN의 time step은 4번
appl -> pple
# appl은 train_X(입력 시퀀스)에 저장
# pple는 train_y(예측해야하는 시퀀스)에 저장
```

<br>

#### 6.1.2.8 문자열을 문장 샘플로 분리

- 이제 15만 8천의 길이를 가진 `text` 문자열로부터 다수의 문장 샘플들로 분리한다.
- 분리하는 방법
  - 문장 샘플의 길이를 정한다.
  - 해당 길이만큼 문자열 전체를 전부 등분한다.

In [80]:
seq_length = 60 # 문장의 길이

n_samples = int(np.floor((len(text) - 1) / seq_length)) # 문자열을 60등분한다. 그러면 즉 총 샘플의 개수
print('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2660


- 만약 문장의 길이를 60으로 한다면 15만 8천을 60으로 나눈 수가 샘플의 수가 된다.
- 여기서는 총 샘플의 수가 2660개 이다.

In [ ]:
train_X = []
train_y = []

for i in range(n_samples):

    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 가져온다.
    X_sample = text[i * seq_length: (i+1) * seq_length]

    # 하나의 문장 샘플에 대해서 정수 인코딩
    X_encoded = [char_to_index[c] for c in X_sample]

    train_X.append(X_encoded)

    # 오른쪽으로 1칸 쉬프트한다.
    y_sample = text[i * seq_length + 1: (i+1) * seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]

    train_y.append(y_encoded)

In [82]:
print(train_X[0])

[50, 38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31]


In [83]:
print(train_y[0])

[38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31, 44]


- `train_y[0]`은 `train_X[0]`에서 오른쪽 한 칸 쉬프트된 문장임을 알 수 있다.

In [84]:
print(train_X[1])

[44, 34, 10, 0, 32, 55, 0, 42, 35, 53, 39, 49, 0, 33, 31, 48, 48, 45, 42, 42, 0, 50, 38, 39, 49, 0, 35, 32, 45, 45, 41, 0, 39, 49, 0, 36, 45, 48, 0, 50, 38, 35, 0, 51, 49, 35, 0, 45, 36, 0, 31, 44, 55, 45, 44, 35, 0, 31, 44, 55]


In [85]:
print(train_y[1])

[34, 10, 0, 32, 55, 0, 42, 35, 53, 39, 49, 0, 33, 31, 48, 48, 45, 42, 42, 0, 50, 38, 39, 49, 0, 35, 32, 45, 45, 41, 0, 39, 49, 0, 36, 45, 48, 0, 50, 38, 35, 0, 51, 49, 35, 0, 45, 36, 0, 31, 44, 55, 45, 44, 35, 0, 31, 44, 55, 53]


- 마찬가지로 `train_y[1]`은 `train_X[1]`에서 오른쪽으로 쉬프트된 문장임을 알 수 있다.

<br>

#### 6.1.2.9 원-핫 인코딩

- 이제 `train_X`와 `train_y`에 대해서 원-핫 인코딩을 수행한다.
- 글자 단위 RNN 에서는 입력 시퀀스에 대해서 워드 임베딩을 하지 않는다.
- 다시 말해 임베딩층(embedding layer)을 사용하지 않을 것이므로, 입력 시퀀스인 `train_X`에 대해서도 원-핫 인코딩을 한다.

In [ ]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

In [87]:
print('train_X의 크기 (shape) : {}'.format(train_X.shape))
print('train_y의 크기 (shape) : {}'.format(train_y.shape))

train_X의 크기 (shape) : (2660, 60, 57)
train_y의 크기 (shape) : (2660, 60, 57)


- `train_X`와 `train_y`의 크기는 `2,660 x 60 x 57` 이다.
  - 샘플의 수(No. of samples) : 2,660개
  - 입력 시퀀스의 길이(input_length) : 60
  - 각 벡터의 차원(input_dim) : 57
    - 원-핫 벡터의 차원은 글자 집합의 크기인 57이어야 하므로 원-핫 인코딩이 수행되었음을 알 수 있다.

$\quad$ ![](https://wikidocs.net/images/page/22886/rnn_image6between7.PNG)


<br>

### 6.1.3 모델 설계하기

#### 6.1.3.1 필요 라이브러리 임포트

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

<br>

#### 6.1.3.2 모델 설계

In [ ]:
model = Sequential()

model.add(LSTM(256, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

<br>

#### 6.1.3.3 모델 컴파일 및 학습

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

```python
Train on 2660 samples
Epoch 1/80
2660/2660 - 43s - loss: 3.0784 - accuracy: 0.1818
Epoch 2/80
2660/2660 - 40s - loss: 2.7360 - accuracy: 0.2478
Epoch 3/80
2660/2660 - 41s - loss: 2.3754 - accuracy: 0.3330

...

Epoch 78/80
2660/2660 - 41s - loss: 0.1846 - accuracy: 0.9553
Epoch 79/80
2660/2660 - 41s - loss: 0.2002 - accuracy: 0.9491
Epoch 80/80
2660/2660 - 41s - loss: 0.1977 - accuracy: 0.9491
<tensorflow.python.keras.callbacks.History at 0x7f09119ee438>
```

<br>

#### 6.1.3.4 문장 생성 함수

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def sentence_generation(model, length):

    # 글자에 대한 랜덤 인덱스 생성
    ix = [np.random.randint(vocab_size)]

    # 랜덤 인덱스로부터 글자 생성
    y_char = [index_to_char[ix[-1]]]

    print(ix[-1], '번 글자', y_char[-1], '로 예측을 시작!')

    # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성
    X = np.zeros((1, length, vocab_size))

    for i in range(length):

        # X[0][i][예측한 글자의 인덱스] = 1. 즉, 예측 글자를 다음 입력 시퀀스에 추가
        X[0][i][ix[-1]] = 1 
        
        print(index_to_char[ix[-1]], end="")

        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])

    return ('').join(y_char)

In [ ]:
sentence_generation(model, 100)

```
8 번 글자 ) 로 예측을 시작!
') and repeat _tis the voice of the sligglows to the choos: that very soon ladge again, down on her ha'
```

<br>

## 6.2 글자 단위 RNN(Char RNN)으로 텍스트 생성하기

- 이번에는 다 대 일(many-to-one) 구조의 RNN을 글자 단위로 학습시키고, 텍스트를 생성해보자.

<br>

### 6.2.1 데이터에 대한 이해와 전처리

#### 6.2.1.1 필요 라이브러리 임포트

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

<br>

#### 6.2.1.2 훈련 데이터

- 다음과 같이 임의로 만든 엉터리 노래 가사가 있다.

In [ ]:
text='''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

<br>

#### 6.2.1.3 하나의 문자열로 저장

- 우선 위의 텍스트에 존재하는 단락 구분을 없애고 하나의 문자열로 재저장한다.

In [96]:
tokens = text.split() # \n 제거
text = ' '.join(tokens)
print(text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


- 단락 구분이 없어지고 하나의 문자열로 재저장된 것을 확인할 수 있다.

<br>

#### 6.2.1.4 글자 집합 생성

- 이제 이로부터 글자 집합을 만들어보자.

In [97]:
char_vocab = sorted(list(set(text)))
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


- 기존의 단어 단위 집합이 아니라 알파벳 또는 구두점 등의 단위의 집합인 글자 집합이 생성됐다.

In [98]:
vocab_size = len(char_vocab)
print('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


<br>

#### 6.2.1.5 글자에 정수 인덱스 부여

In [99]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


- 이번 실습의 글자 집합의 경우, 훈련 데이터에 등장한 알파벳의 대, 소문자를 구분하고 구두점과 공백을 포함시켰다.

<br>

#### 6.2.1.6 문장 샘플 생성

- 이제 훈련에 사용할 문장 샘플들을 만들어보자.
- 여기서는 RNN을 이용한 생성한 텍스트 챕터와 유사하게 데이터를 구성한다.
- 다만, 단위가 글자 단위라는 점이 다르다.
- 예를 들어, 훈련 데이터에 "student"라는 단어가 있고, 입력 시퀀스의 길이를 5라고 한다.  
$\rightarrow$ 입력 시퀀스와 예측해야 하는 글자는 다음과 같이 구성된다.

```python
# Example) 5개의 입력 글자 시퀀스로부터 다음 글자 시퀀스를 예측. 즉, RNN의 time step은 5번
stude -> n
tuden -> t
```

- 여기서는 입력 시퀀스의 길이, 즉 모든 샘플들의 길이가 10이 되도록 데이터를 구성한다.
- 예측 대상이 되는 글자도 필요하므로 우선 길이가 11이 되도록 데이터를 구성한다.

In [100]:
length = 11
sequences = []

for i in range(length, len(text)):
    seq = text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)

print('총 훈련 샘플의 수 : %d' % (len(sequences)))

총 훈련 샘플의 수 : 426


In [101]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

- 첫 번째 문장이었던 "I get on with life as a programmer"가 10개의 샘플로 분리된 것을 확인할 수 있다.
- 다른 문장들에 대해서도 `sequences`에 모두 저장되어 있다.

<br>

#### 6.2.1.7 전체 데이터 정수 인코딩

- 이제 앞서 만든 `char_to_index`를 사용하여 전체 데이터에 대해서 정수 인코딩을 수행한다.

In [ ]:
X = []

for line in sequences:
    temp_X = [char_to_index[char] for char in line]
    X.append(temp_X)

In [103]:
for line in X[:5]:
    print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


<br>

#### 6.2.1.8 예측 대상 글자 분리

- 이제 예측 대상인 글자를 분리시켜주는 작업을 한다.
- 모든 샘플 문장에 대해서 맨 마지막 글자를 분리시켜준다.

In [ ]:
sequences = np.array(X)

X = sequences[:, :-1]
y = sequences[:, -1]

In [105]:
for line in X[:5]:
    print(line)

[ 8  0 16 14 28  0 24 23  0 31]
[ 0 16 14 28  0 24 23  0 31 18]
[16 14 28  0 24 23  0 31 18 28]
[14 28  0 24 23  0 31 18 28 17]
[28  0 24 23  0 31 18 28 17  0]


In [106]:
print(y[:5])

[18 28 17  0 21]


- 앞서 출력한 5개의 샘플에서 각각 맨 뒤의 글자였던 `[18 28 17  0 21]`이 별도로 분리되어 `y`에 저장되었다.

<br>

#### 6.2.1.9 원-핫 인코딩

- 이제 `X`와 `y`에 대해서 원-핫 인코딩을 수행한다.

In [ ]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [108]:
print(X.shape)

(426, 10, 33)


- 현재 `X`의 크기는 `426 x 10 x 33` 이다.
  - 426 : 샘플의 수 (No. of samples)
  - 10 : 입력 시퀀스의 길이 (`input_length`)
  - 33 : 각 벡터의 차원(`input_dim`), 원-핫 벡터의 차원은 글자 집합의 크기인 33이어야 한다.
  
$\quad$ ![](https://wikidocs.net/images/page/22886/rnn_image6between7.PNG)

<br>

### 6.2.2 모델 설계하기

#### 6.2.2.1 필요 라이브러리 임포트

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

<br>

#### 6.2.2.2 모델 설계

- LSTM을 사용
- 은닉 상태의 크기 : 80
- 출력층에 단어 집합의 크기만큼의 뉴런을 배치하여 모델을 설계한다.
- 출력층의 활성화 함수 : 소프트맥스 함수

In [ ]:
model = Sequential()

model.add(LSTM(80, input_shape=(X.shape[1], X.shape[2]))) # X.shape[1] : 10, X.shape[2] : 33
model.add(Dense(vocab_size, activation='softmax'))

<br>

#### 6.2.2.3 모델 컴파일 및 훈련

- 손실 함수 : 크로스 엔트로피 함수
- 총 100번의 에포크 수행

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=2)

```python
Train on 426 samples
Epoch 1/100
426/426 - 1s - loss: 3.4670 - accuracy: 0.1432
Epoch 2/100
426/426 - 0s - loss: 3.3343 - accuracy: 0.1972
Epoch 3/100
426/426 - 0s - loss: 3.0707 - accuracy: 0.1972

...

Epoch 98/100
426/426 - 0s - loss: 0.1532 - accuracy: 0.9812
Epoch 99/100
426/426 - 0s - loss: 0.1513 - accuracy: 0.9789
Epoch 100/100
426/426 - 0s - loss: 0.1458 - accuracy: 0.9812
<tensorflow.python.keras.callbacks.History at 0x7f0894cc6860>
```

<br>

#### 6.2.2.4 문장 생성 함수

- 문장을 생성하는 함수 `sentence_generation`을 만들어서 문장을 생성해보자.

In [ ]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
    # model : 모델
    # char_to_index : 인덱스 정보
    # seq_length ; 문장 길이
    # seed_text : 초기 시퀀스
    # n : 반복 횟수

    init_text = seed_text # 문장 생성에 사용할 초기 시퀀스
    sentence = ''

    for _ in range(n):

        # 현재 시퀀스에 대한 정수 인코딩
        encoded = [char_to_index[char] for char in seed_text]

        # 데이터에 대한 패딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')

        encoded = to_categorical(encoded, num_classes=len(char_to_index))

        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장
        result = model.predict_classes(encoded, verbose=0)

        for char, index in char_to_index.items():
            if index == result: # 만약 예측한 글자와 인덱스가 동일한 글자가 있다면
                break # 해당 글자가 예측 글자이므로 break

        seed_text = seed_text + char # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        sentence = sentence + char # 예측 글자를 문장에 저장

        # for문이므로 이 작업을 다시 반복

    sentence = init_text + sentence

    return sentence

In [113]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to use words about beer. But when I stap my tra


- 두 개의 문장이 출력되었는데 훈련 데이터에서는 연속적으로 나온 적이 없는 두 문장임에도 모델이 임의로 잘 생성해낸 것 같다.